# NPM Complex Network Analysis (Top-N)

Bu notebook, NPM ekosistemindeki paketleri yönlü bir ağ (Dependent → Dependency) olarak modelleyip, merkeziyet metrikleri ile yapısal riski analiz eder.

Hedefler:
- Top-N (ör. 200) paket listesini indirmek (downloads’a göre)
- `Dependent → Dependency` yönünde bir `DiGraph` kurmak
- In-degree ve betweenness centrality hesaplamak
- Sonuçları tablo ve dosya olarak kaydetmek

Not: Büyük N (örn. 20k) için bağımlılık çekme ve betweenness hesaplaması maliyetlidir; bu notebook N=200 civarında etkileşimli çalışmaya uygundur.

## 1) Kurulum
Gerekli bağımlılıkları yükleyelim. Eğer sanal ortam kullanıyorsanız, önce etkinleştirin.

In [ ]:
%pip install -q -r requirements.txt

## 2) İçe Aktarım ve Parametreler
Var olan yardımcı fonksiyonları `src/analyze_npm_network.py` içinden kullanacağız.

In [ ]:
from pathlib import Path
import sys
import os

# src altını import yoluna ekle
sys.path.append(str(Path('src').resolve()))

import networkx as nx
from analyze_npm_network import (
    fetch_top_packages,
    build_dependency_graph,
    compute_metrics,
    save_edges,
    save_metrics,
    save_report,
)

# Tercihen sabit Top 200 listesi
INPUT_LIST = Path('data/top_200.txt')
USE_INPUT_LIST = INPUT_LIST.exists()
TOP_N = 200  # fetching fallback
OUTDIR = Path('results')
OUTDIR.mkdir(parents=True, exist_ok=True)
OUTDIR

## 3) Top-N Paketleri Getir
Önce indirme sayısına göre sıralı Top-N listeyi çekiyoruz. Ecosystems API öncelikli, gerekirse yedek kaynaklar kullanılır.

In [ ]:
if USE_INPUT_LIST:
    from analyze_npm_network import read_list
    top_packages = read_list(INPUT_LIST)
    print(f'Loaded {len(top_packages)} from {INPUT_LIST}')
else:
    top_packages = fetch_top_packages(TOP_N)
    print(f'Fetched {len(top_packages)} from API(s)')
len(top_packages), top_packages[:20]

## 4) Ağın Kurulması (Directed)
Düğümler: paketler; Kenarlar: `Dependent → Dependency`.

In [ ]:
G, top_set = build_dependency_graph(top_packages)
G.number_of_nodes(), G.number_of_edges()

## 5) Merkeziyet Metrikleri
In-degree (pakete gelen kenar sayısı) ve betweenness (en kısa yollardaki aracı rol) hesaplanır.

In [ ]:
in_deg, btw = compute_metrics(G)
# Özet
nodes = G.number_of_nodes()
edges = G.number_of_edges()
nodes, edges

## 6) İlk 15 – In-Degree ve Betweenness
Top listeleri hızlıca görelim.

In [ ]:
top_in_all = sorted(in_deg.items(), key=lambda kv: kv[1], reverse=True)[:15]
top_btw_all = sorted(btw.items(), key=lambda kv: kv[1], reverse=True)[:15]

top_in_top = sorted(((n, in_deg.get(n, 0)) for n in top_set), key=lambda kv: kv[1], reverse=True)[:15]
top_btw_top = sorted(((n, btw.get(n, 0.0)) for n in top_set), key=lambda kv: kv[1], reverse=True)[:15]

top_in_all, top_btw_all, top_in_top, top_btw_top

## 7) Sonuçları Kaydet
Kenarlıklar, metrikler ve kısa raporu dosyaya yazalım.

In [ ]:
save_edges(G, OUTDIR / 'edges.csv')
save_metrics(in_deg, btw, top_set, OUTDIR / 'metrics.csv')
save_report(in_deg, btw, top_set, OUTDIR / 'report.md')
(OUTDIR / 'top_packages.txt').write_text(os.linesep.join(top_packages), encoding='utf-8')
sorted(OUTDIR.iterdir())

## 8) (Opsiyonel) Küçük Görselleştirme
Büyük grafikleri çizmek zor olabilir; küçük bir alt-ağ (ör. en yüksek in-degree düğümünün ego grafiği) çizilebilir.

In [ ]:
%pip install -q matplotlib

In [ ]:
# En yüksek in-degree sahibi düğüm
hub, _ = max(in_deg.items(), key=lambda kv: kv[1]) if in_deg else (None, None)
hub

In [ ]:
# Hub etrafında küçük bir alt-ağ çizimi
try:
    import matplotlib.pyplot as plt
    if hub is not None:
        neighbors = set(G.predecessors(hub)) | set(G.successors(hub)) | {hub}
        H = G.subgraph(neighbors).copy()
        pos = nx.spring_layout(H, seed=42)
        plt.figure(figsize=(8, 6))
        nx.draw_networkx_nodes(H, pos, node_size=200, alpha=0.8)
        nx.draw_networkx_edges(H, pos, alpha=0.3, arrows=True, arrowsize=10)
        nx.draw_networkx_labels(H, pos, font_size=8)
        plt.axis('off')
        plt.show()
    else:
        print('Görselleştirilecek hub bulunamadı.')
except ModuleNotFoundError:
    print('matplotlib bulunamadı; görselleştirme atlandı.')

## 10) Top 200 Ağ Çizimi (Tüm Düğümler Dahil)Tüm grafiği (Top 200 + bağımlılıkları) çizelim. Top 200 düğümlerini farklı renkte gösteriyoruz ve düğüm boyutlarını in-degree ile ölçekliyoruz.

In [ ]:
import mathimport matplotlib.pyplot as plt# Yerleşim ve görselleştirme parametreleriN = G.number_of_nodes()pos = nx.spring_layout(G, k=1/math.sqrt(max(N,1)), seed=42)# Düğüm boyutları: in-degree tabanlımax_in = max(in_deg.values()) if in_deg else 1node_sizes = [ 100 + (400 * (in_deg.get(n,0) / max_in if max_in else 0)) for n in G.nodes() ]# Renk: Top 200 turuncu, diğerleri mavinode_colors = [ "tab:orange" if n in top_set else "tab:blue" for n in G.nodes() ]plt.figure(figsize=(12, 9), dpi=150)nx.draw_networkx_edges(G, pos, arrows=False, width=0.5, alpha=0.25)nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.85, linewidths=0.2, edgecolors="#444444")plt.title("NPM Top 200 Bağımlılık Ağı (Tümü)")plt.axis("off")out_png = OUTDIR / "network_full_top200.png"out_svg = OUTDIR / "network_full_top200.svg"plt.tight_layout()plt.savefig(out_png)plt.savefig(out_svg)out_png, out_svg

## 11) Top 200 Kohortu Ağ Çizimi (Sadece Top 200)Top 200 düğümlerin oluşturduğu indüklenmiş alt-ağı çizelim.

In [ ]:
H = G.subgraph(top_set).copy()HN = H.number_of_nodes() or 1posH = nx.spring_layout(H, k=1/math.sqrt(HN), seed=42)max_in_H = max((in_deg.get(n,0) for n in H.nodes()), default=1)node_sizes_H = [ 120 + (480 * (in_deg.get(n,0) / max_in_H if max_in_H else 0)) for n in H.nodes() ]node_colors_H = [ "tab:orange" for _ in H.nodes() ]plt.figure(figsize=(10, 8), dpi=150)nx.draw_networkx_edges(H, posH, arrows=False, width=0.6, alpha=0.35)nx.draw_networkx_nodes(H, posH, node_size=node_sizes_H, node_color=node_colors_H, alpha=0.9, linewidths=0.2, edgecolors="#444444")# İsteğe bağlı: az sayıda düğüm varsa etiketleri gösterif H.number_of_nodes() <= 60:    nx.draw_networkx_labels(H, posH, font_size=6)plt.title("NPM Top 200 Bağımlılık Ağı (Sadece Top 200)")plt.axis("off")out_png2 = OUTDIR / "network_top200_only.png"out_svg2 = OUTDIR / "network_top200_only.svg"plt.tight_layout()plt.savefig(out_png2)plt.savefig(out_svg2)out_png2, out_svg2